In [338]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
to_run = True

In [339]:
#Loading the data
path = "C:/Users/abels/ITU/Privacy/Final_Proj/data/"
results = pd.read_excel(path+"public_data_resultsG.xlsx")
public =  pd.read_excel(path+"public_data_registerG.xlsx")
private =  pd.read_excel(path+"private_dataG.xlsx")

In [340]:
public.head()

,name,sex,dob,zip,citizenship,marital_status,last_voted
0,"Zeng, Darren",Male,1998-04-28,2200,Denmark,Never married,2
1,"Mian, Lauren",Female,2003-11-01,2100,Denmark,Never married,2
2,"Shumpert, Chieloka",Female,1998-10-07,2300,Denmark,Never married,1
3,"Martinez, Joslyn",Female,2004-02-15,2200,Denmark,Never married,1
4,"Lewis, Sydney",Female,2000-04-20,2200,Bulgaria,Never married,1


In [341]:
#preparation of survey data
survey = private.groupby(["evote","party"]).size().reset_index(name="count")

data = {
    "evote" : survey["evote"].values,
    "party" : survey["party"].values,
    "count" : survey["count"].values
}
df = pd.DataFrame(data)
df_survey = df.pivot_table(index ="evote",columns="party",values="count",fill_value=0)
df_survey["Total"] = df_survey.sum(axis=1)
df_survey = df_survey.reset_index().rename_axis(None, axis=1)
df_survey = df_survey[['evote', 'Red', 'Green', 'Invalid vote', 'Total']]
totals = df_survey[['Red', 'Green', 'Invalid vote', 'Total']].sum()
# totals["evote"] = "Total"
# df_survey = pd.concat([df_survey,totals], ignore_index=True)
df_survey.loc["column_total"] = df_survey.sum(axis=0)
df_survey.head()



,evote,Red,Green,Invalid vote,Total
0,0.0,44.0,79.0,4.0,127.0
1,1.0,31.0,38.0,4.0,73.0
column_total,1.0,75.0,117.0,8.0,200.0


In [342]:
#preparation of results

if to_run:
    results.loc["poll_votes"] = results.iloc[0:4,1:].sum()

    results.iloc[6,0] ="poll_votes"
    results = results.iloc[[4,5,6]]
    results = results.iloc[[0,2,1]]
    results = results.reset_index()
    results = results[["Unnamed: 0",	"Red",	"Green",	"Invalid ballots",	"Total"]]
    to_run = False
results

,Unnamed: 0,Red,Green,Invalid ballots,Total
0,E-votes,136.0,243.0,10.0,389.0
1,poll_votes,223.0,444.0,18.0,685.0
2,Total,359.0,687.0,28.0,1074.0


In [343]:
df_survey.head()

,evote,Red,Green,Invalid vote,Total
0,0.0,44.0,79.0,4.0,127.0
1,1.0,31.0,38.0,4.0,73.0
column_total,1.0,75.0,117.0,8.0,200.0


(A) Is there a significant difference between the political preferences as expressed in the survey and the election results for both electronic and polling station votes?

In [344]:
# TODO
"""
    scale down results by 200/1074 DONE
    get expectations for results df
    do chi square of independence between the two groups
"""
scaler = float(results.iloc[-1,-1]/df_survey.iloc[-1,-1])
for i in range(results.shape[0]):
    for j in range(1,5):
        results.iloc[i,j] = results.iloc[i,j] / scaler      #Rescaling results to same size as survey
results

,Unnamed: 0,Red,Green,Invalid ballots,Total
0,E-votes,25.325885,45.251397,1.862197,72.439479
1,poll_votes,41.527002,82.681564,3.351955,127.560521
2,Total,66.852886,127.932961,5.214153,200.000000


In [345]:
#get expectations for results df
expected_results = results
for i in range(1,4):
    for j in range(0,2):
        grand_total = expected_results.iloc[2,4]
        row_total = expected_results.iloc[j,4]
        column_total = expected_results.iloc[2,i]
        expectation = (row_total*column_total)/grand_total
        expected_results.iloc[j,i] = expectation
expected_results = expected_results.reindex([1,0,2])

        
# expected_results.iloc[j,4] row total
# expected_results.iloc[2,i] column total
expected_results

,Unnamed: 0,Red,Green,Invalid ballots,Total
1,poll_votes,42.638945,81.595976,3.325600,127.560521
0,E-votes,24.213941,46.336985,1.888553,72.439479
2,Total,66.852886,127.932961,5.214153,200.000000


In [346]:
df_survey

,evote,Red,Green,Invalid vote,Total
0,0.0,44.0,79.0,4.0,127.0
1,1.0,31.0,38.0,4.0,73.0
column_total,1.0,75.0,117.0,8.0,200.0


In [347]:
df_chi = pd.DataFrame([[1,2,3],[4,5,6]], index = ["poll","evote"],columns=["Red","Green","Invalid"])
for i in range(1,4):
    for j in range(0,2):
        Eji = expected_results.iloc[j,i]
        Oji = df_survey.iloc[j,i]
        val = ((Oji-Eji)**2)/Eji
        df_chi.iloc[j,i-1] = val

df_chi

C:\Users\abels\AppData\Local\Temp\ipykernel_20356\897840964.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.043445494452612525' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_chi.iloc[j,i-1] = val
C:\Users\abels\AppData\Local\Temp\ipykernel_20356\897840964.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0825909765068581' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_chi.iloc[j,i-1] = val
C:\Users\abels\AppData\Local\Temp\ipykernel_20356\897840964.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1367618125674349' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_chi.iloc[j,i-1] = val


,Red,Green,Invalid
poll,0.043445,0.082591,0.136762
evote,1.901822,1.499997,2.360649


In [348]:
#testing of results
df_survey.iloc[1,1]
# df_chi.iloc[0,3]
# expected_results.iloc[0,4]

((4-1.888553)**2)/1.888553

2.3606477730881794

In [349]:
df_chi = df_chi[["Red","Green"]] 
print(df_chi) 
row_sums = df_chi.sum(axis=1)
row_sums
degree_of_freedom = 1


            Red     Green
poll   0.043445  0.082591
evote  1.901822  1.499997


with significance level $\alpha$ = 0.05, the critical threshhold for a chi square test is 3.841.

For the poll votes we find no significant difference between the results and the survey data with a value of 0.12.
For the evotes we find a higher value 3.40, but still not high enough to reject the null hypothesis and state that the findings are significantly different between the survey and the results data.

 .

(B) Is there a significant difference between political preferences of the voters depending on their demographic attributes recorded in the survey (that is, age, gender, education level…)?

In [ ]:
#Logistic Regression
# data transformation

df_survey = private[private["party"].isin(["Red","Green"])] # removing invalid votes
df_survey.head()
df_survey.shape

(192, 9)

In [ ]:
#Making binary party column
df_survey["party_bin"] = df_survey["party"].apply(lambda x: 1 if x == "Green" else 0)
df_survey.head()

C:\Users\abels\AppData\Local\Temp\ipykernel_20356\1281275867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survey["party_bin"] = df_survey["party"].apply(lambda x: 1 if x == "Green" else 0)


,name,sex,evote,dob,zip,education,citizenship,marital_status,party,party_bin
0,"Lewis, Samantha",Female,1,1975-07-30,2300,Vocational Education and Training (VET),Denmark,Married/separated,Green,1
1,"Moon, Ethan",Male,0,1937-10-21,2200,Primary education,Denmark,Widowed,Green,1
2,"Hoskins, Dawit",Male,0,1973-11-05,2100,Vocational Education and Training (VET),Denmark,Married/separated,Red,0
3,"Lowe, Brandon",Male,0,1977-04-19,2200,Masters programmes,Denmark,Married/separated,Red,0
4,"Sakamoto, Bruce",Male,1,1991-03-23,2200,Masters programmes,Ukraine,Married/separated,Green,1


In [447]:
#Making age groups column
now = datetime.date.today()
df_survey["age"] = df_survey["dob"].apply(lambda x: now.year - x.year)
df_survey["age_group"] = df_survey["age"].apply(lambda x: 0 if x <= 35 else(1 if x <= 45 else(2 if x <= 55 else(3 if x <= 65 else 4))))
df_survey.groupby(["age_group"]).size().head()
df_survey.head()

C:\Users\abels\AppData\Local\Temp\ipykernel_20356\4293181944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survey["age"] = df_survey["dob"].apply(lambda x: now.year - x.year)
C:\Users\abels\AppData\Local\Temp\ipykernel_20356\4293181944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survey["age_group"] = df_survey["age"].apply(lambda x: 0 if x <= 35 else(1 if x <= 45 else(2 if x <= 55 else(3 if x <= 65 else 4))))


,name,sex,evote,dob,zip,education,citizenship,marital_status,party,party_bin,age,age_group
0,"Lewis, Samantha",Female,1,1975-07-30,2300,Vocational Education and Training (VET),Denmark,Married/separated,Green,1,49,2
1,"Moon, Ethan",Male,0,1937-10-21,2200,Primary education,Denmark,Widowed,Green,1,87,4
2,"Hoskins, Dawit",Male,0,1973-11-05,2100,Vocational Education and Training (VET),Denmark,Married/separated,Red,0,51,2
3,"Lowe, Brandon",Male,0,1977-04-19,2200,Masters programmes,Denmark,Married/separated,Red,0,47,2
4,"Sakamoto, Bruce",Male,1,1991-03-23,2200,Masters programmes,Ukraine,Married/separated,Green,1,33,0


In [454]:
#Making binary gender column
df_survey["sex_bin"] = df_survey["sex"].apply(lambda x: 0 if x =="Male" else 1) 


C:\Users\abels\AppData\Local\Temp\ipykernel_20356\3149349340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survey["sex_bin"] = df_survey["sex"].apply(lambda x: 0 if x =="Male" else 1)


In [ ]:
#making numerical education column
education_mapping = {
    "primary education" : 0,
    "Bachelors programmes": 2,
    "Masters programmes": 3,
    "Not stated": 0,
    "PhD programmes": 3,
    "Short cycle higher education": 0,
    "Upper secondary education": 0,
    "Vocational Education and Training (VET)": 1,
    "Vocational bachelors educations": 2 
}
df_survey["education_numeric"] = df_survey["education"].replace(education_mapping)
df_survey.groupby(["education_numeric"]).size()

C:\Users\abels\AppData\Local\Temp\ipykernel_20356\2358511778.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_survey["education_numeric"] = df_survey["education"].replace(education_mapping)
C:\Users\abels\AppData\Local\Temp\ipykernel_20356\2358511778.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survey["education_numeric"] = df_survey["education"].replace(education_mapping)


education_numeric
0    65
1    64
2    39
3    24
dtype: int64

Encodings:

-party:
0 Red
1 Green

-age:
0 19-33
1 33-44 
2 45-55
3 56-65
4 65+

-sex
0 Male
1 Female

-education
0 primary
1 vocational
2 bachelor
3 master+

In [459]:
df = df_survey[["party_bin","age_group","sex_bin","education_numeric"]]
df

,party_bin,age_group,sex_bin,education_numeric
0,1,2,1,1
1,1,4,0,0
2,0,2,0,1
3,0,2,0,3
4,1,0,0,3
...,...,...,...,...
195,1,2,1,0
196,1,0,1,1
197,1,4,0,0
198,0,2,1,2


In [352]:
def sigmoid(z):
    return 1/ (1 + np.exp(-z))

(C) Is there a significant difference between voter’s choice of the voting channel (that is, if they decide to vote either online or in person) depending on their demographic attributes recorded in the survey?